In [11]:
import sympy
from sympy import sin, cos, simplify
from sympy import symbols as syms
from sympy.matrices import Matrix
from sympy.utilities.lambdify import lambdastr


# state, state derivative, and control variables
q_0, q_1, q_2, qdot_0, qdot_1, qdot_2, qddot_0, qddot_1, qddot_2, f = syms('q_0 q_1 q_2 qdot_0 qdot_1 qdot_2 qddot_0 qddot_1 qddot_2 f')

# parameters
r_1, r_2, m_c, m_1, m_2, g = syms('r_1 r_2 m_c m_1 m_2 g')

p = Matrix([r_1, r_2, m_c, m_1, m_2, g])      # parameter vector

q = Matrix([q_0, q_1, q_2])                   # generalized positions
qdot = Matrix([qdot_0, qdot_1, qdot_2])       # time derivative of q
qddot = Matrix([qddot_0, qddot_1, qddot_2])   # time derivative of qdot

# To calculate time derivatives of a function f(q), we use:
# df(q)/dt = df(q)/dq * dq/dt = df(q)/dq * qdot

# kinematics:
p_c = Matrix([q_0, 0])
p_1 = p_c + r_1/2 * Matrix([cos(q_1), sin(q_1)])
p_2 = p_c + r_1 * Matrix([cos(q_1), sin(q_1)]) + r_2/2 * Matrix([cos(q_1+q_2), sin(q_1+q_2)])

v_c = p_c.jacobian(Matrix([q_0])) * Matrix([qdot_0])
v_1 = p_1.jacobian(Matrix([q_0, q_1])) * Matrix([qdot_0, qdot_1])
v_2 = p_2.jacobian(Matrix([q_0, q_1, q_2])) * Matrix([qdot_0, qdot_1, qdot_2])

K_c = m_c * v_c.T*v_c / 2
K_1 = m_1 * v_1.T*v_1 / 2
K_2 = m_2 * v_2.T*v_2 / 2

P_1 = Matrix([m_1 * g * p_1[1]])
P_2 = Matrix([m_2 * g * p_2[1]])

# dynamics:

# Lagrangian L=sum(K)-sum(P)
L = K_c + K_1 + K_2 - P_1 - P_2

# first term in the Euler-Lagrange equation
partial_L_by_partial_q = L.jacobian(Matrix([q])).T

# inner term of the second part of the Euler-Lagrange equation
partial_L_by_partial_qdot = L.jacobian(Matrix([qdot]))

# second term (overall, time derivative) in the Euler-Lagrange equation
# applies the chain rule
d_inner_by_dt = partial_L_by_partial_qdot.jacobian(Matrix([q])) * qdot + partial_L_by_partial_qdot.jacobian(Matrix([qdot])) * qddot

# Euler-Lagrange equation
lagrange_eq =  partial_L_by_partial_q - d_inner_by_dt + Matrix([f,0,0])

# substitude parameters with numerical values to get simpler equations
lagrange_eq = lagrange_eq.subs({r_1:1, r_2:1, m_c:5, m_1:1, m_2:1, g:9.81});

# solve the lagrange equation for qddot and simplify
print("Calculations take a while...")
r = sympy.solvers.solve(simplify(lagrange_eq), Matrix([qddot]))

qddot_0 = r[qddot_0];
qddot_1 = r[qddot_1];
qddot_2 = r[qddot_2];

# draw the equations that we can copy&paste to the ode
print('qddot_0 = {}\n'.format(qddot_0));
print('qddot_1 = {}\n'.format(qddot_1));
print('qddot_2 = {}\n'.format(qddot_2));

Calculations take a while...
qddot_0 = (-4.0*f*cos(2.0*q_2) + 6.0*f + 4.0*qdot_1**2*cos(q_1) + qdot_1**2*cos(q_1 - q_2) - qdot_1**2*cos(q_1 + 2.0*q_2) + 2.0*qdot_1*qdot_2*cos(q_1 - q_2) + qdot_2**2*cos(q_1 - q_2) - 29.43*sin(2.0*q_1) + 9.81*sin(2.0*q_1 + 2.0*q_2))/(3.0*cos(2.0*q_1) - 22.0*cos(2.0*q_2) - cos(2.0*q_1 + 2.0*q_2) + 34.0)

qddot_1 = (8.0*f*sin(q_1) - 4.0*f*sin(q_1 + 2.0*q_2) + 3.0*qdot_1**2*sin(2.0*q_1) + 23.0*qdot_1**2*sin(q_2) + 22.0*qdot_1**2*sin(2.0*q_2) + qdot_1**2*sin(2.0*q_1 + q_2) + 46.0*qdot_1*qdot_2*sin(q_2) + 2.0*qdot_1*qdot_2*sin(2.0*q_1 + q_2) + 23.0*qdot_2**2*sin(q_2) + qdot_2**2*sin(2.0*q_1 + q_2) - 490.5*cos(q_1) + 215.82*cos(q_1 + 2.0*q_2))/(3.0*cos(2.0*q_1) - 22.0*cos(2.0*q_2) - cos(2.0*q_1 + 2.0*q_2) + 34.0)

qddot_2 = -((100.0*qdot_1**2*sin(q_2) + 981.0*cos(q_1 + q_2))*(-9.0*(sin(q_1) + 0.333333333333333*sin(q_1 + q_2))**2 + 28.0*cos(q_2) + 42.0) + 0.5*(200.0*qdot_1*qdot_2*sin(q_2) + 100.0*qdot_2**2*sin(q_2) - 2943.0*cos(q_1) - 981.0*cos(q_1 + q_2))*(25.